In [7]:
import os
import re
import shutil
from urllib.parse import urljoin
from urllib.parse import unquote

import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
dir_item = '/home/jake/Documents/News_Item/'
dir_proj = dir_item + 'fukushima/'
dir_tableau = dir_proj + 'tableau_data/'

dir_pdf = './pdf/'
# dir_clean = './clean/'

if not os.path.exists(dir_pdf):
    os.makedirs(dir_pdf)

# if not os.path.exists(dir_clean):
#     os.makedirs(dir_clean)

In [9]:
# a = 1
# f'{a:>03}'

'001'

In [10]:
# for dir_name in os.listdir(dir_pdf):
#     shutil.move(dir_pdf+dir_name, dir_pdf+f'{dir_name:>03}')

In [3]:
def get_hrefs(page=1):
    url = f'https://www.mfds.go.kr/brd/m_604/list.do?page={page}&srchFr=&srchTo=&srchWord=&srchTp=&itm_seq_1=0&itm_seq_2=0&multi_itm_seq=0&company_cd=&company_nm='
    r = requests.get(url)
    soup = bs(r.text, 'html.parser')
    files = soup.select('.bbs_icon_filedown')
    hrefs = [f['href'] for f in files]
    return hrefs
    

def download_pdf(page, hrefs):
    url_base = 'https://www.mfds.go.kr/brd/m_604/'
    for href in hrefs:

        url_download = urljoin(url_base, href)
        r = requests.get(url_download)

        obj = re.search(r'.+filename=\"(.+)\"', r.headers['Content-Disposition'])

        if obj:
            title_quoted = obj.group(1)
            title = unquote(title_quoted)
        else:
            seq = href.split('&')[1]
            title = page + '_' + seq
        
        dir_pdf_page = f'{dir_pdf}{page}/'
        if not os.path.exists(dir_pdf_page):
            os.makedirs(dir_pdf_page)
            
#         dir_clean_page = f'{dir_clean}{page}/'
#         if not os.path.exists(dir_clean_page):
#             os.makedirs(dir_clean_page)
            
        with open(dir_pdf_page + title, 'wb') as f:
            f.write(r.content)

        command = f'mutool clean {dir_pdf_page}{title} {dir_pdf_page}{title}'
        os.system(command)

In [11]:
for page in range(212, 430):
    hrefs = get_hrefs(page)
    if not hrefs:
        continue
    download_pdf(page, hrefs)

In [ ]:
# 211: error